# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [108]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc, udf, col, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType
from pyspark.sql import Window

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json" 
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName('Data Wrangling Quiz') \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [4]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [15]:
user_log.take(1)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046')]

In [58]:
user_log.show(1)

+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|       artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+------+
|Showaddywaddy|Logged In|  Kenneth|     M|          112|Matthews|232.93342| paid|Charlotte-Concord...|   PUT|NextSong|1509380319284|     5132|Christmas Tears W...|   200|1513720872284|"Mozilla/5.0 (Win...|  1046|
+-------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-------------+---------+------

# Question 1

Which page did user id "" (empty string) NOT visit?

In [7]:
user_log.describe("userId").show()

+-------+------------------+
|summary|            userId|
+-------+------------------+
|  count|             10000|
|   mean|1442.4413286423842|
| stddev| 829.8909432082613|
|    min|                  |
|    max|               999|
+-------+------------------+



In [19]:
user_log.describe("status").show()

+-------+------------------+
|summary|            status|
+-------+------------------+
|  count|             10000|
|   mean|          202.8984|
| stddev|18.041791154505876|
|    min|               200|
|    max|               404|
+-------+------------------+



In [24]:
# TODO: write your code to answer question 1

set(user_log.select('page').where(user_log.userId == "").collect())

{Row(page='About'), Row(page='Help'), Row(page='Home'), Row(page='Login')}

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [60]:
# TODO: use this space to explore the behavior of the user with an empty string
set(user_log.select('registration').where(user_log.userId == "").collect())

{Row(registration=None)}

# Question 3

How many female users do we have in the data set?

In [33]:
user_log.describe('gender').show()

+-------+------+
|summary|gender|
+-------+------+
|  count|  9664|
|   mean|  null|
| stddev|  null|
|    min|     F|
|    max|     M|
+-------+------+



In [34]:
user_log.groupBy(["gender"]).count().show()

+------+-----+
|gender|count|
+------+-----+
|     F| 3820|
|  null|  336|
|     M| 5844|
+------+-----+



In [54]:
user_log.where(user_log.gender == 'F').select(count('gender')).show()

+-------------+
|count(gender)|
+-------------+
|         3820|
+-------------+



In [30]:
# TODO: write your code to answer question 3
len(set(user_log.select(["gender", 'userId']).where(user_log.gender == 'F').collect()))

462

In [63]:
user_log.filter(user_log.gender == 'F') \
    .select('gender', 'userId') \
    .dropDuplicates() \
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [81]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist', 'song') \
    .groupby('Artist') \
    .agg(count('Artist').alias('NumArtist')) \
    .sort(desc('NumArtist')) \
    .show()  #1

+--------------------+---------+
|              Artist|NumArtist|
+--------------------+---------+
|            Coldplay|       83|
|       Kings Of Leon|       69|
|Florence + The Ma...|       52|
|            BjÃÂ¶rk|       46|
|       Dwight Yoakam|       45|
|       Justin Bieber|       43|
|      The Black Keys|       40|
|         OneRepublic|       37|
|                Muse|       36|
|        Jack Johnson|       36|
|           Radiohead|       31|
|        Taylor Swift|       29|
|Barry Tuckwell/Ac...|       28|
|          Lily Allen|       28|
|               Train|       28|
|           Daft Punk|       27|
|           Metallica|       27|
|          Nickelback|       27|
|          Kanye West|       26|
|Red Hot Chili Pep...|       24|
+--------------------+---------+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [87]:
isHome = udf(lambda x: int(x == 'home'), returnType=IntegerType())

In [114]:
windowval = Window.partitionBy("userId") \
            .orderBy(desc("ts")) \
            .rangeBetween(Window.unboundedPreceding, 0)

In [115]:
# TODO: write your code to answer question 5
cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
            .select('userId', 'page', 'ts') \
            .withColumn('homeVisit', isHome(col('page'))) \
            .withColumn('period', Fsum('homeVisit').over(windowval))

In [116]:
cusum.show(5)

+------+--------+-------------+---------+------+
|userId|    page|           ts|homeVisit|period|
+------+--------+-------------+---------+------+
|  1436|NextSong|1513783259284|        0|     0|
|  1436|NextSong|1513782858284|        0|     0|
|  2088|    Home|1513805972284|        0|     0|
|  2088|NextSong|1513805859284|        0|     0|
|  2088|NextSong|1513805494284|        0|     0|
+------+--------+-------------+---------+------+
only showing top 5 rows



In [117]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupby('userId', 'period') \
    .agg(count('period').alias('visits')) \
    .agg(avg('visits')) \
    .show()

+----------------+
|     avg(visits)|
+----------------+
|9.87810650887574|
+----------------+

